In [1]:
import pandas as pd
import datetime as dt
import json
import os
import time
import math
import numpy as np
import talib
import random
from technical import *

In [2]:
rtdlist = [ f for f in os.listdir('../rtd/') if f.endswith(".json") ]
ohlclist = [ f for f in os.listdir('../ohlc/') if f.endswith(".csv") ]
def telegram(msg):
    import requests
    print(msg)
    url="https://api.telegram.org/bot1362017119:qwertyuiop"+str(msg)
    #requests.get(url)

In [3]:
book={'Time':[],'id':[],'Pair':[],'Side':[],'Qty':[], 'Trail':[],'TP':[],'SL':[],'PL':[],'OLTP':[],'CLTP':[],'CTime':[]}
pairs=['EUR/USD', 'GBP/USD', 'USD/CHF','AUD/USD']
stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}
maxtrade=10000
max_spread_percent=0.00018
demo=True
base_qty = 75
is_mis=True
bookpath='./book.csv'
trade_duration=60*60*10
waiting_time=60*60*4
max_concurrent_trade=5
max_profit_day=60
max_loss_day=-80
max_loss_per_trade=-5.2
suspended_pair=[]
max_open_trade_per_pair=1
max_concurrent_pair=1
suspension_limit=-5
max_profit_per_day_per_pair=40
suspnsion_time=20*60*60
atr_value=0.5
exp_day_next=False
trade_stop=False
def dynamic_bookpath():
    path= './CASINO FIXED REV.csv'
    return path

In [4]:
def modification_date():
    return True
    t = os.path.getmtime('../rtd/EURUSD.json')
    return dt.datetime.fromtimestamp(t)>dt.datetime.now()-dt.timedelta(seconds=3600)
def tradeTime(offset=False,starttime=None,endtime=None):
    time_now=dt.datetime.now().hour*100+dt.datetime.now().minute
    if offset==True:
        if not endtime>time_now>starttime:
            return False
    if modification_date() ==  False:
        return False
    if dt.date.today().weekday() in [5,6]:
        return False
    return True

In [5]:
def convert_ohlc(data,time='240Min'):
    try:
        data.index=data.date
    except:
        pass
    ohlc_dict = {                                                                                                             
        'Open':'first',                                                                                                    
        'High':'max',                                                                                                       
        'Low':'min',                                                                                                        
        'Close': 'last',                                                                                                    
    }
    return data.resample(time, closed='right').agg(ohlc_dict).dropna(how='any')

def ohlc(pair='EURUSD',tme='240Min'):
    for i in range(20):
        try:
            pair=pair.replace('/','')
            data=pd.read_csv('../ohlc/'+pair+'.csv')
            data.date=pd.to_datetime(data.date)
#             return data
            return convert_ohlc(data,time=tme)
        except:
            time.sleep(0.2)
    return    

def rtd(pair='EURUSD'):
    for i in range(20):
        try:
            pair=pair.replace('/','')
            with open('../rtd/'+pair+'.json', 'r') as fp:
                data= json.load(fp)
            data['ltp']=(data['Bid']+data['Ask'])/2
            data['Spread']=data['Ask']-data['Bid']
            return data
        except:
            time.sleep(0.2)
    return

def resume_book():
    data=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    book={'Time':[],'id':[],'Pair':[],'Side':[],'Qty':[], 'Trail':[],'TP':[],'SL':[],'PL':[],'OLTP':[],'CLTP':[],'CTime':[]}
    for key in list(data.keys()):
        for i in range(len(data[key])):
            try:
                book[key].append(dt.datetime.fromisoformat(data[key][i]))
            except:
                book[key].append(data[key][i])
    return book


def current_pl(data,pair=None):
    if not pair == None:
        data=data[data['Pair']==pair]
        data=data.reset_index(drop=True)
    data.Time=pd.to_datetime(data.Time)
    data.index=data.Time
    data= data[dt.date.today():]
    data=data.reset_index(drop=True)
    data
    pl=0
    for profit in data.PL:
        if not profit == 'False':
            pl=pl+float(profit)
    openOrder=data
    openOrder=data[data['CLTP']==False]
    try:
        openOrderx=data[data['CLTP']=='False']
    except:
        openOrderx=data[data['CLTP']==False]
    if len(openOrderx) >len(openOrder):
        openOrder=openOrderx
    for i in openOrder.index:
        if openOrder.Side[i] == 'Buy':
            pl=pl+(float(rtd(openOrder.Pair[i])['Bid']-float(openOrder.OLTP[i])))*openOrder.Qty[i]
        else:
            pl=pl+(float(openOrder.OLTP[i])-rtd(openOrder.Pair[i])['Ask'])*openOrder.Qty[i]
    return pl

In [6]:
def stop_for_day(offset=0):
    pl=current_pl(pd.DataFrame(book))
    if pl-offset>max_profit_day or offset+pl<max_loss_day:
        return True
    return False

In [7]:
def momentum(data):
    fastk, fastd = talib.STOCHRSI(data.Close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)
    if int((fastk.iloc[-1]))<int(fastd.iloc[-1]) and int(fastd.iloc[-1]) < int(fastd.iloc[-2]):
        return True
    elif int((fastk.iloc[-1]))>int(fastd.iloc[-1]) and int(fastd.iloc[-1]) > int(fastd.iloc[-2]):
        return False
    return 1

In [8]:
def tp_sl_calculation(data,cdl=10):
    data['BSize']=abs(data.Open-data.Close)
    data['HLSize']=abs(data.High-data.Low)
    data['OHSize']=abs(data.High-data.Open)
    data['OLSize']=abs(data.Open-data.Low)
    bc_avg = (np.average(data.BSize[-cdl:])+min(data.BSize[-cdl:]))/2
    sell  = bc_avg+np.average(data.OLSize[-cdl:])
    low_min = min(data.OLSize[-cdl:])
    low_max = max(data.OLSize[-cdl:])
    high_max = max(data.OHSize[-cdl:])
    high_min = min(data.OHSize[-cdl:])
    buy = bc_avg+np.average(data.OHSize[-cdl:])
    data = data.dropna()
    return [buy,sell,low_min,high_min,low_max,high_max]



def signal(data,ltp,maximum=False):
    ltp = ltp['ltp']
    buy,sell,low_min,high_min,low_max,high_max = tp_sl_calculation(data,cdl=10)
    buy=buy-max_spread_percent*ltp
    sell=sell-max_spread_percent*ltp
    open_price=data.Open[-1]
    high_price=data.High[-1]
    low_price=data.Low[-1]
    open_prev=data.Open[-2]
    high_prev=data.High[-2]
    low_prev=data.Low[-2]
    close_prev=data.Close[-2]
    mid_prev = (open_prev+close_prev)/2
#     print(mid_prev,ltp)
    only = None
    if maximum == True:
        if buy>sell:
            maximum='Buy'
        else:
            maximum='Sell'
    macd, macdsignal, macdhist = talib.MACD(data.Close, fastperiod=12, slowperiod=26, signalperiod=9)
    if (only==None or only == 'Buy') and (ltp < (mid_prev)) and (macd.iloc[-2]<macdsignal.iloc[-2],macdhist.iloc[-2]>macdhist.iloc[-3]):# and (high_price < ltp+buy):
        return [True,True,buy,low_max]
    if (only==None or only == 'Sell') and (ltp > (mid_prev)) and (macd.iloc[-2]>macdsignal.iloc[-2],macdhist.iloc[-2]<macdhist.iloc[-3]):# and (low_price > ltp-sell):
        return [True,False,sell,high_max]
    return [False,False,False,False]

In [9]:
def place(pair,side,tp,sl,qty=1,offset=0):
    global book
    try:
        book=resume_book()
        pass
    except:
        pass
    openOrdert=pd.DataFrame(book)
    if trade_stop:
        print('day pl limit reached')
        return False    
    if len(openOrdert)>=maxtrade:
        #print('Max trades already exist')
        return False
    try:
        openOrderx=openOrdert[openOrdert['CLTP']=='False']
    except:
        openOrderx=openOrdert[openOrdert['CLTP']==False]
    openOrder=openOrdert[openOrdert['CLTP']==False]
    if len(openOrderx)>len(openOrder):
        openOrder=openOrderx
    if len(openOrder)>=max_concurrent_trade:
        #print('Many Trades running')
        return
    try:
        openOrder=openOrder[openOrder['Pair']==pair]
        if len(openOrder)>=max_open_trade_per_pair:
            return False
    except:
        pass
    #########################################################
    if side=='Buy':
        ltp=rtd(pair)['Ask']
    else:
        ltp=rtd(pair)['Bid']
    if demo==False:
        if not is_mis == True:
            pass
        else:
            pass
        if not response['status'] == 'success':
            print(response)
            return 
        order_id=response['data']['oms_order_id']
    else:
        try:
            order_id=max(book['id'])+1
        except:
            order_id=1
    
    #########################################################
    book['id'].append(order_id)
    book['Time'].append(dt.datetime.now())
    book['Pair'].append(pair)
    book['Qty'].append(qty)
    book['Side'].append(side)
    book['Trail'].append(False)
    book['PL'].append(False)
    book['OLTP'].append(float(ltp)+offset)
    book['CLTP'].append(False)
    book['CTime'].append(False)
    book['TP'].append(ltp+float(tp) if side=='Buy' else ltp-float(tp))
    tp=ltp+float(tp) if side=='Buy' else ltp-float(tp)
    book['SL'].append(ltp-float(sl) if side=='Buy' else ltp+float(sl))
    pd.DataFrame(book).to_csv(dynamic_bookpath(),index=False)
    msg = 'BOT1 '+'id: '+str(order_id)+' '+side+' '+pair+' '+' Price: '+str(ltp)+' Qty: '+str(qty)+' TP:'+str(tp)
    telegram(msg)  


In [10]:
def closeTrade(tk):
    global book
    try:
        book=resume_book()
        pass
    except:
        pass
    openOrder=pd.DataFrame(book)
    index=tk-1
#     openOrder=openOrder[openOrder['id']==int(tk)]
    
    side=book['Side'][index]
    pair=book['Pair'][index]
    cltp=book['CLTP'][index]
    qty=int(book['Qty'][index])
    if not (cltp == False or cltp == 'False'):
        #print('Trade already closed')
        return
    if side=='Buy':
        ltp=rtd(pair)['Bid']
        book['PL'][index]=(ltp-float(book['OLTP'][index]))*qty
        pl=(ltp-float(book['OLTP'][index]))*qty
    else:
        ltp=rtd(pair)['Ask']
        book['PL'][index]=(float(book['OLTP'][index])-ltp)*qty
        pl=(float(book['OLTP'][index])-ltp)*qty
    
    book['CLTP'][index]=ltp
    book['CTime'][index]=dt.datetime.now()
    pd.DataFrame(book).to_csv(dynamic_bookpath(),index=False)
    telegram('BOT1: Trade closed '+str(side)+' '+str(pair)+' Price: '+str(ltp)+' Quantity: '+str(qty)+' Profit/Loss: '+str(pl))
    try:
        book=resume_book()
        pass
    except:
        pass


In [11]:

def closeAll(pair=None):
    global book
    try:
        resume_book()
        openOrderx=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        openOrderx=pd.DataFrame(book)
    openOrder=openOrderx[openOrderx['PL'] == False]
    try:
        openOrdert=openOrderx[openOrderx['PL'] == 'False']
        if len(openOrder)<len(openOrdert):
            openOrder=openOrdert
    except:
        pass
    if not pair==None:
        openOrder=openOrder[openOrder['Pair']==pair]
    for i in openOrder.id:
        closeTrade(i)
    try:
        book=resume_book()
    except:
        pass

def check_suspension(pair='EUR/USD'):
    global book
    try:
        book=resume_book()
        pass
    except:
        pass
    data=pd.DataFrame(book)
    if len(data)==0:
        return True
    data_temp=data#data[data['PL']!=False].reset_index(drop=True)
    data_temp=data_temp[data_temp['Pair']==pair].reset_index(drop=True)
    pl=current_pl(data,pair)
#     print(pl)
    if pl < suspension_limit:
        last_placed=pd.to_datetime(data_temp.Time).iloc[-1]
        if last_placed+dt.timedelta(seconds=suspnsion_time) > dt.datetime.now():
            closeAll(pair)
            stopped_pair[pair]=True
            return False
    return True

def check_daily_limit():
    global book
    try:
        resume_book()
        data=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        data=pd.DataFrame(book)
    if len(data)==0:
        return
    for pair in pairs:
#         print(current_pl(data,pair))
        if current_pl(data,pair)>max_profit_per_day_per_pair:
            stopped_pair[pair]=True
            #closeAll(pair)
    if current_pl(data,pair)<max_profit_per_day_per_pair:
        stopped_pair[pair]=False


In [12]:
def checkClose():
    global book
    try:
        book=resume_book()
        openOrdert=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        openOrdert=pd.DataFrame(book)
    try:
        openOrderx=openOrdert[openOrdert['CLTP']=='False']
    except:
        openOrderx=openOrdert[openOrdert['CLTP']==False]
    openOrder=openOrdert[openOrdert['CLTP']==False]
    if len(openOrderx)>len(openOrder):
        openOrder=openOrderx
    index=list(openOrder.index)
    if len(index)==0:
        return
    for i in index:
        tp = float(book['TP'][i])
        sl = float(book['SL'][i])
        side = book['Side'][i]
        otime = book['Time'][i]
        pair = book['Pair'][i]
        oltp = float(book['OLTP'][i])
        tk = book['id'][i]
        if side=='Buy':
            ltp=rtd(pair)['Bid']
        else:
            ltp=rtd(pair)['Ask']
        try:
            if type(otime)==str:
                otime=dt.datetime.strptime(otime, '%Y-%m-%d %H:%M:%S.%f')
        except:
            pass
        if dt.timedelta(seconds=trade_duration)+otime <= dt.datetime.now():
            print('Time Close')
            closeTrade(tk)
            
        high=max(tp,sl)
        low=min(tp,sl)
        if side=='Buy':
            if pair=='USD/JPY':
                pl=(ltp-oltp)*10
            else:
                pl=(ltp-oltp)*1000
        else:
            if pair=='USD/JPY':
                pl=(oltp-ltp)*10
            else:
                pl=(oltp-ltp)*1000
        if (low >= ltp or high <= ltp or pl<max_loss_per_trade):
            if side == 'Buy':
                if high <= ltp:
                    print('TP Close')
                else:
                    print('SL Close')
            else:
                if high <= ltp:
                    print('SL Close')
                else:
                    print('TP Close')
            closeTrade(tk)

In [13]:
def checkPlace():
    global book
    pass_trade=False
    side_restricted=None
    try:
        book=resume_book()
        openOrder=pd.read_csv(dynamic_bookpath(),parse_dates=True)
    except:
        openOrder=pd.DataFrame(book)
    for pair in pairs:
        time_now=dt.datetime.now()#+dt.timedelta(seconds=waiting_time)
        time_now=time_now.replace(minute=0,second=0,microsecond=0)
        data=ohlc(pair)
        try:
            last_placed=openOrder[openOrder['Pair']==pair]
        except:
            last_placed=[]        
        if len(last_placed)>0:
            last_placed=last_placed.reset_index(drop=True)
            temp_restricted=last_placed[last_placed['CLTP']==False]
            temp_restricted=temp_restricted.reset_index(drop=True)
            if len(temp_restricted)>=max_concurrent_pair:
                pass_trade=True
                
            if len(temp_restricted)>0:
                side_restricted = 'Sell' if temp_restricted.Side.iloc[-1]=='Buy' else 'Buy'
            last_placed=last_placed.Time.iloc[-1]
            try:
                if type(last_placed)==str:
                    last_placed=dt.datetime.strptime(last_placed, '%Y-%m-%d %H:%M:%S.%f')
            except:
                pass
            last_placed=last_placed.replace(minute=0,second=0,microsecond=0)
            if last_placed==time_now or last_placed >= time_now:
                last_placed=False
        else:
            last_placed=True
        if not last_placed==False:
            last_placed=True
        now = signal(ohlc(pair),rtd(pair))
        if last_placed and rtd(pair)['Spread']/(rtd(pair)['ltp'])<max_spread_percent and pass_trade == False:# and check_suspension(pair) and stopped_pair[pair]==False:
            if now[1] == True and (not side_restricted == 'Buy'):
                #print('Buy Trade place')
                tp=0.0019
                sl=0.002
                if pair == 'USD/JPY':
                    place(pair,'Buy',sl,tp,qty=1000)
                else:
                    place(pair,'Buy',sl,tp,qty=1000)
            elif (not side_restricted == 'Sell'):
                #print('Sell trade Placed')
                tp=0.0019
                sl=0.002
                if pair == 'USD/JPY':
                    place(pair,'Sell',sl,tp,qty=10)
                else:
                    place(pair,'Sell',sl,tp,qty=1000)

In [14]:
try:
    book=resume_book()
    pass
except:
    pass
pd.DataFrame(book)

,Time,id,Pair,Side,Qty,Trail,TP,SL,PL,OLTP,CLTP,CTime
0,2020-10-14 15:26:47.224334,1,EUR/USD,Sell,1000,False,1.17402,1.17792,0.8100000000001994,1.17602,1.17521,2020-10-15 01:26:50.368261
1,2020-10-14 15:26:50.622264,2,GBP/USD,Sell,1000,False,1.30093,1.30483,2.1400000000000308,1.30293,1.30079,2020-10-14 16:24:31.232209
2,2020-10-14 15:26:51.706312,3,USD/CHF,Buy,1000,False,0.91450,0.91060,0.6099999999999994,0.91250,0.9131100000000001,2020-10-15 01:26:55.695768
3,2020-10-14 16:24:33.994224,4,GBP/USD,Sell,1000,False,1.29881,1.30271,-1.9199999999999215,1.30081,1.30273,2020-10-14 17:04:37.795457
4,2020-10-14 16:36:03.248670,5,AUD/USD,Buy,1000,False,0.71810,0.71420,-1.9299999999999875,0.71610,0.71417,2020-10-14 22:37:01.472124
5,2020-10-14 17:04:40.399988,6,GBP/USD,Sell,1000,False,1.30066,1.30456,1.440000000000108,1.30266,1.30122,2020-10-15 03:04:43.483797
6,2020-10-15 01:22:38.802606,7,AUD/USD,Buy,1000,False,0.71648,0.71258,-1.9400000000000528,0.71448,0.71254,2020-10-15 02:52:25.034918
7,2020-10-15 01:26:52.164117,8,EUR/USD,Sell,1000,False,1.17312,1.17702,2.0000000000002243,1.17512,1.1731200000000002,2020-10-15 08:05:05.223264
8,2020-10-15 01:26:59.746020,9,USD/CHF,Buy,1000,False,0.91521,0.91131,2.0299999999999763,0.91321,0.9152399999999999,2020-10-15 08:48:43.740993
9,2020-10-15 03:04:46.980507,10,GBP/USD,Buy,1000,False,1.30321,1.29931,-1.9299999999997648,1.30121,1.2992800000000002,2020-10-15 07:00:27.679853


In [15]:
# while True:
#     while tradeTime():
#         checkPlace()
#         checkClose() 
#         check_daily_limit()
        

In [ ]:
while True:
    while tradeTime():
        try:
            checkPlace()
            checkClose() 
            check_daily_limit()
        except Exception as e:
            print(str(e))
            f = open("log.txt", "a")
            f.write(str(e)+"\n")
            f.close()
            time.sleep(1)
            pass
        
    if not tradeTime():
        closeAll()
        stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}
        
#         if stop_for_day():
#             trade_stop=True
#             print('stop for day close all')
#             stopped_pair={'EUR/USD':True, 'GBP/USD':True, 'USD/CHF':True,'AUD/USD':True}
#             closeAll()
        
#         if tradeTime(offset=True,starttime=500,endtime=530):
#             trade_stop=False
#             stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}

Time Close
BOT1: Trade closed Buy EUR/USD Price: 1.17818 Quantity: 1000 Profit/Loss: 6.520000000000081
TP Close
Time Close
BOT1: Trade closed Sell GBP/USD Price: 1.30025 Quantity: 1000 Profit/Loss: -7.189999999999808
SL Close
Time Close
BOT1: Trade closed Sell USD/CHF Price: 0.90998 Quantity: 1000 Profit/Loss: 4.979999999999984
TP Close
Time Close
BOT1: Trade closed Sell AUD/USD Price: 0.71086 Quantity: 1000 Profit/Loss: -0.7200000000000539


/root/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1164: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


BOT1 id: 55 Sell EUR/USD  Price: 1.17818 Qty: 1000 TP:1.17618
BOT1 id: 56 Sell GBP/USD  Price: 1.3001200000000002 Qty: 1000 TP:1.2981200000000002
BOT1 id: 57 Buy USD/CHF  Price: 0.90999 Qty: 1000 TP:0.91199


In [ ]:
current_pl(pd.DataFrame(book),'EUR/USD')

In [ ]:
tradeTime()

In [ ]:
while True:
    while tradeTime():
        checkPlace()
        checkClose() 
        check_daily_limit()
        

In [ ]:
stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}


In [ ]:
stopped_pair

In [ ]:
trade_stop=False
stopped_pair={'EUR/USD':False, 'GBP/USD':False, 'USD/CHF':False,'AUD/USD':False}


In [ ]:
current_pl(pd.DataFrame(book))